In [2]:
import numpy as np

opcodeInput=np.loadtxt("input_day15.txt",delimiter=",",dtype=np.int64)

class OpCodeManager:
    def __init__(self, opcode):
        self.opcode = opcode
        self.relativeBaseIdx=0
    def __getitem__(self, idx):
        if idx >= self.opcode.size:
            self.opcode.resize(idx+1, refcheck=False) # fill rest with zeros
        return self.opcode[idx]
    def __setitem__(self, idx, value):
        if idx >= self.opcode.size:
            self.opcode.resize(idx+1, refcheck=False) # fill rest with zeros
        self.opcode[idx] = value

def decodeCommand(command):
    code=command%100
    p1Mode=int(command/100)%10
    p2Mode=int(command/1000)%10
    p3Mode=int(command/10000)%10
    return [code, p1Mode, p2Mode, p3Mode]

def getArgument(opcode, mode, parameter):
    if mode == 0:
        # position mode
        return opcode[parameter]
    elif mode == 1:
        # immediate mode
        return parameter
    else:
        # relative mode
        return opcode[opcode.relativeBaseIdx+parameter]
    
def getWritePosition(opcode, mode, parameter):
    if mode == 0:
        # position mode
        return parameter
    elif mode == 1:
        # immediate mode
        print("ERROR! Immediate mode not intended for writing")
        print(mode, parameter, opcode)
    else:
        # relative mode
        return opcode.relativeBaseIdx+parameter

In [57]:
def getPositionOffsetFromCommand(command):
    if command == 1:
        # north
        return [0,-1]
    elif command == 2:
        # south
        return [0,1]
    elif command == 3:
        # west
        return [-1,0]
    elif command == 4:
        # east
        return [1,0]
    else:
        return [0,0]
    
def getNeighborhoodStatus(position, locationsMap):
    neighborhoodStatus = []
    for i in range(1,5):
        neighborhoodPos = tuple(position+getPositionOffsetFromCommand(i))
        if neighborhoodPos in locationsMap:
            neighborhoodStatus.append(locationsMap[neighborhoodPos])
        else:
             neighborhoodStatus.append(-1);
    return neighborhoodStatus

def getNeighborhoodStatusTuple(position, locationsMap):
    neighborhoodStatus = []
    for i in range(1,5):
        neighborhoodPos = tuple(position+getPositionOffsetFromCommand(i))
        if neighborhoodPos in locationsMap:
            neighborhoodStatus.append((neighborhoodPos,locationsMap[neighborhoodPos]))
        else:
             neighborhoodStatus.append((neighborhoodPos,-1));
    return neighborhoodStatus

class RepairDroid:
    def __init__(self, opcodeMgr):
        self.opcode = opcodeMgr
        self.outputList=[]
        self.locations={}
        self.input=0
        self.position=np.array([0,0])
        
    def clearOutput(self):
        self.outputList=[]
    
    def processOutput(self):
        #print(self.outputList)
        if len(self.outputList) == 0: return
        status = self.outputList[0] # there should only be a single element in list
        positionOffset = np.array(getPositionOffsetFromCommand(self.input))
        statusLocation = self.position + positionOffset
        if status == 0:
            # wall
            self.locations[tuple(statusLocation)] = 0
        elif status == 1 or status == 2:
            # move to free field or oxygen
            self.locations[tuple(statusLocation)] = status
            self.position += positionOffset
        else:
            print("Unknown status! ", status)
        self.clearOutput()
        #print(self.position,self.locations)
        

    def visualize(self):
        xMax,yMax,xMin,yMin=0,0,0,0
        for loc in self.locations:
            xMax=max(xMax,loc[0])
            yMax=max(yMax,loc[1])
            xMin=min(xMin,loc[0])
            yMin=min(yMin,loc[1])
        #enlarge by 1
        xMin -= 1
        yMin -= 1
        xMax += 1
        yMax += 1
        #print(xMin,xMax,yMin,yMax)
        # screen: > = x, v = y
        screen=np.full((yMax+1-yMin,xMax+1-xMin)," ")
        for loc in self.locations:
            locType=self.locations[loc]
            locStr=' '
            if locType == 0:
                locStr= '#'
            elif locType == 1:
                locStr = '.'
            elif locType == 2:
                locStr = 'O'
            screen[loc[1]-yMin,loc[0]-xMin] = locStr
        screen[self.position[1]-yMin,self.position[0]-xMin] = 'D'
        outputStr=''
        for j,row in enumerate(screen):
            for elem in row:
                outputStr += elem
            outputStr += '\n'
        print(outputStr)
    
    
    def run(self, idx):
    
        while True:

            decoded = decodeCommand(self.opcode[idx])
            
            #if len(self.outputList) >= 2 and self.outputList[-2] == -1 and self.outputList[-1] == 0:
            #    print(idx, decoded)

            if decoded[0] == 99:
                print("Finished!")
                return 0
            elif decoded[0] == 1:
                #add p1 p2 p3
                p1 = self.opcode[idx+1]
                p2 = self.opcode[idx+2]
                p3 = self.opcode[idx+3]
                arg1 = getArgument(self.opcode,decoded[1],p1)
                arg2 = getArgument(self.opcode,decoded[2],p2)
                arg3 = getWritePosition(self.opcode,decoded[3],p3)
                self.opcode[arg3] = arg1 + arg2
                #if arg1 + arg2 in [98,59,85]: print(str(arg1) + " + " + str(arg2) + " = " + str(arg1+arg2))
                idx+=4
            elif decoded[0] == 2:
                #multiply p1 p2 p3
                p1 = self.opcode[idx+1]
                p2 = self.opcode[idx+2]
                p3 = self.opcode[idx+3]
                arg1 = getArgument(self.opcode,decoded[1],p1)
                arg2 = getArgument(self.opcode,decoded[2],p2)
                arg3 = getWritePosition(self.opcode,decoded[3],p3)
                self.opcode[arg3] = arg1 * arg2
                #if arg1 * arg2 in [98,59,85]: print(str(arg1) + " * " + str(arg2) + " = " + str(arg1*arg2))
                idx+=4
            elif decoded[0] == 3:
                #input p1
                p1 = self.opcode[idx+1]
                arg1 = getWritePosition(self.opcode,decoded[1],p1)
                # break here and wait for updated input from extern
                yield 1
                var = self.input
                if var == 0:
                    # stop
                    return 0
                self.opcode[arg1] = np.int64(var)
                idx+=2
            elif decoded[0] == 4:
                #output p1
                p1 = self.opcode[idx+1]
                arg1 = getArgument(self.opcode,decoded[1],p1)
                #print("Output: ", arg1)
                self.outputList.append(arg1)
                idx+=2
            elif decoded[0] == 5:
                #jump-if-true p1 p2
                p1 = self.opcode[idx+1]
                p2 = self.opcode[idx+2]
                arg1 = getArgument(self.opcode,decoded[1],p1)
                arg2 = getArgument(self.opcode,decoded[2],p2)
                if arg1 != 0:
                    idx=arg2
                else:      
                    idx=idx+3
            elif decoded[0] == 6:
                #jump-if-false p1 p2
                p1 = self.opcode[idx+1]
                p2 = self.opcode[idx+2]
                arg1 = getArgument(self.opcode,decoded[1],p1)
                arg2 = getArgument(self.opcode,decoded[2],p2)
                if arg1 == 0:
                    idx=arg2
                else:      
                    idx+=3
            elif decoded[0] == 7:
                #less-than p1 p2 p3
                p1 = self.opcode[idx+1]
                p2 = self.opcode[idx+2]
                p3 = self.opcode[idx+3]
                arg1 = getArgument(self.opcode,decoded[1],p1)
                arg2 = getArgument(self.opcode,decoded[2],p2)
                arg3 = getWritePosition(self.opcode,decoded[3],p3)
                if arg1 < arg2:
                    self.opcode[arg3]=1
                else:      
                    self.opcode[arg3]=0
                idx+=4
            elif decoded[0] == 8:
                #equals p1 p2 p3
                p1 = self.opcode[idx+1]
                p2 = self.opcode[idx+2]
                p3 = self.opcode[idx+3]
                arg1 = getArgument(self.opcode,decoded[1],p1)
                arg2 = getArgument(self.opcode,decoded[2],p2)
                arg3 = getWritePosition(self.opcode,decoded[3],p3)
                if arg1 == arg2:
                    self.opcode[arg3]=1
                else:      
                    self.opcode[arg3]=0
                idx+=4
            elif decoded[0] == 9:
                # change relative base p1
                p1 = self.opcode[idx+1]
                arg1 = getArgument(self.opcode,decoded[1],p1)
                self.opcode.relativeBaseIdx += arg1
                idx+=2
            else:
                print("Invalid code ", self.opcode, idx, self.opcode[idx], decoded)
                return

In [49]:
#task 1
def convertInput(inputStr):
    if inputStr == 'w':
        return 1
    elif inputStr == 's':
        return 2
    elif inputStr == 'a':
        return 3
    elif inputStr == 'd':
        return 4
    elif inputStr == 'q':
        return 0
    else:
        return convertInput(input("Repeat input: "))

# bad..
import random
class Droidbot:
    def __init__(self, direction):
        self.direction=direction
        self.explore=False
        
    def nextInput(self,droid):
        pos = droid.position
        neighborhoodStatus = getNeighborhoodStatus(pos, droid.locations)
        newDirection=0
        unknownDirections=[]
        freeDirections=[]
        for i,status in enumerate(neighborhoodStatus):
            if status == -1:
                unknownDirections.append(i+1)
            if status == 1:
                freeDirections.append(i+1)
                
        if len(unknownDirections) > 0:
                # pick a random unknwon direction
                newDirection = random.choice(unknownDirections)
                self.explore=False
        elif len(freeDirections) > 0:
            if len(freeDirections) == 1:
                # we found a dead end
                self.explore=True
            if neighborhoodStatus[self.direction-1] != 0 and not self.explore:
                #go in same direction
                newDirection = self.direction
            else:
                # pick a random known but free direction
                newDirection = random.choice(freeDirections)
        
        else:
            #dunno?? just pick some direction..
            newDirection = random.choice([1,2,3,4])
        self.direction=newDirection
        #print(newDirection)
        return newDirection

# good!
class HeatMapBot:
    def __init__(self):
        self.heat = 0
        self.heatMap = {(0,0):0}
        
    def getNeighborhoodHeat(self,pos):
        neighborhoodStatus = []
        for i in range(1,5):
            neighborhoodPos = tuple(pos+getPositionOffsetFromCommand(i))
            if neighborhoodPos in self.heatMap:
                neighborhoodStatus.append(self.heatMap[neighborhoodPos])
            else:
                 neighborhoodStatus.append(-1);
        return neighborhoodStatus
    
    def nextInput(self,droid):
        pos = droid.position
        posTup = tuple(pos)
        if posTup in self.heatMap:
            self.heat = self.heatMap[posTup]
        else:
            self.heatMap[posTup] = self.heat
        neighborhoodStatus = getNeighborhoodStatus(pos, droid.locations)
        neighborhoodHeat = self.getNeighborhoodHeat(pos)
        newDirection=0
        unknownDirections=[]
        heatInUnknownDirections=[]
        freeDirections=[]
        heatInFreeDirections=[]
        for i,status in enumerate(neighborhoodStatus):
            if status == -1:
                unknownDirections.append(i+1)
                heatInUnknownDirections.append(neighborhoodHeat[i])
            if status == 1:
                freeDirections.append(i+1)
                heatInFreeDirections.append(neighborhoodHeat[i])
                
        if len(unknownDirections) > 0:
            newDirection = unknownDirections[heatInUnknownDirections.index(min(heatInUnknownDirections))]
        elif len(freeDirections) > 0:
            newDirection = freeDirections[heatInFreeDirections.index(min(heatInFreeDirections))]
        else:
            #dunno?? just pick some direction..
            newDirection = random.choice([1,2,3,4])
        self.heat += 1
        return newDirection
    

In [51]:
from IPython.display import clear_output
botCyclesBeforeRestart = 3000

currentLocations={(0,0):1}

opcodeMgr=OpCodeManager(opcodeInput.copy())
droid = RepairDroid(opcodeMgr)
droid.locations = currentLocations.copy()
#bot = Droidbot(random.choice([1,2,3,4]))
bot = HeatMapBot()
currentCycle = 0
for i,step in enumerate(droid.run(0)):
    droid.processOutput()
    droid.visualize()
    #droid.input=convertInput(input("New direction: "))
    droid.input=bot.nextInput(droid)
    #game.resetTiles()
    currentCycle += 1
    if currentCycle > botCyclesBeforeRestart:
        currentLocations = droid.locations.copy()
        currentCycle = 0
        break
    clear_output(True)

                                           
  ### ####### ################### #######  
 #...#.......#...................#.......# 
 #.#.#.###.#.#.#######.#########.#######.# 
 #.#...#.#.#.#.#...#...#.......#...#.....# 
 #.#####.#.#.#.#.#.#.###.###### ##.#.###.# 
 #...#...#.#.#.#.#...#.........#...#.#...# 
  ##.#.###.#.###.#############.#.###.#.##  
 #...#.#...#.#...#.......#.....#.....#.#.# 
 #.###.#.###.#.#.#.#####.#.###########.#.# 
 #.....#.#.....#.#.#...#.#...........#.#.# 
  ####.#.#######.#.###.#.#.#######.###.#.# 
 #.#...#...#...#.#.#...#.#.......#...#...# 
 #.#.#####.#.#.###.#.###.#######.###.###.# 
 #...#...#...#...#.#.#...#.....#.#.#...#.# 
 #.###.#########.#.#.#.###.#.###.#.###.#.# 
 #.....#.........#.#.#...#.#...#.#.#...#.# 
  ####.#.###.#####.#.###.## ##.#.#.#.###.# 
 #.#...#.#...#...#.#...#...#...#...#.....# 
 #.#.###.#####.#.#.###.###.#.#.###.######  
 #.#.#.#.......#...#...#...#.#.#...#.....# 
 #.#.#.#############.###.###.###.###.###.# 
 #.#.......#.........#.#.#.....#

In [83]:
def getValidNeighborsList(pos, locationsMap):
    validNeighborsList = []
    neighborhoodState = getNeighborhoodStatusTuple(pos,locationsMap)
    #print(neighborhoodState)
    for e in neighborhoodState:
        #print(e,e[1])
        if e[1] == 1 or e[1] == 2:
            validNeighborsList.append(e[0])
    return validNeighborsList

def checkStatus(pos,locationsMap):
    if locationsMap[tuple(pos)] == 2:
        return True
    return False

def visualizeDistanceMap(distanceMap):
    xMax,yMax,xMin,yMin=0,0,0,0
    for loc in distanceMap:
        xMax=max(xMax,loc[0])
        yMax=max(yMax,loc[1])
        xMin=min(xMin,loc[0])
        yMin=min(yMin,loc[1])
    #enlarge by 1
    xMin -= 1
    yMin -= 1
    xMax += 1
    yMax += 1
    #print(xMin,xMax,yMin,yMax)
    # screen: > = x, v = y
    screen=np.full((yMax+1-yMin,xMax+1-xMin),"    ")
    for loc in distanceMap:
        screen[loc[1]-yMin,loc[0]-xMin] = "."+str(distanceMap[loc])
    outputStr=''
    for j,row in enumerate(screen):
        for elem in row:
            outputStr += elem
        outputStr += '\n'
    print(outputStr)



def getShortestDistanceToOxygen(locationsMap):
    distanceMap={}
    currentDistance=0
    activePositions=[np.array([0,0])]
    nextPositions=[]
    while True:
        #print("Current distance: ", currentDistance)
        #print(distanceMap)
        #print(activePositions)
        for pos in activePositions:
            if tuple(pos) in distanceMap:
                # already found, with smaller distance
                continue
            if checkStatus(pos, locationsMap):
                # found!
                visualizeDistanceMap(distanceMap)
                return currentDistance
            
            distanceMap[tuple(pos)] = currentDistance
            
            neighbors = getValidNeighborsList(pos,locationsMap)
            for neighbor in neighbors:
                nextPositions.append(np.array(list(neighbor)))
        # swap
        if len(nextPositions) == 0:
            # not found
            visualizeDistanceMap(distanceMap)
            return -1
        activePositions,nextPositions = nextPositions,[]
        currentDistance += 1


In [84]:
distance = getShortestDistanceToOxygen(currentLocations)

                                                                                                                                                                    
    .188.187.186    .180.179.178.177.176.177.178    .216.215.214.213.212.211.210.209.208.209.210.211.212.213.214.215.216.217.218    .248.247.246.245.244.243.242    
    .189    .185    .181            .175    .179    .217                            .207                                    .219                            .241    
    .190    .184.183.182    .212    .174    .180    .218    .196.197.198    .204.205.206    .300.301.302.303.304.305.306    .220.221.222    .236.237.238.239.240    
    .191                    .211    .173    .181    .219    .195    .199    .203            .299                                    .223    .235            .241    
    .192.193.194    .208.209.210    .172    .182    .220    .194    .200.201.202    .300.299.298.297.296.295.294.293.292    .226.225.224    .234    .244.243.242    
          

In [85]:
# task 1, finally....
print(distance)

404


In [103]:
# task 2
def getValidNeighborsList(pos, locationsMap):
    validNeighborsList = []
    neighborhoodState = getNeighborhoodStatusTuple(pos,locationsMap)
    #print(neighborhoodState)
    for e in neighborhoodState:
        #print(e,e[1])
        if e[1] == 1:
            validNeighborsList.append(e[0])
    return validNeighborsList


def visualizeOxygenMap(oxygenMap):
    xMax,yMax,xMin,yMin=0,0,0,0
    for loc in oxygenMap:
        xMax=max(xMax,loc[0])
        yMax=max(yMax,loc[1])
        xMin=min(xMin,loc[0])
        yMin=min(yMin,loc[1])
    #enlarge by 1
    xMin -= 1
    yMin -= 1
    xMax += 1
    yMax += 1
    #print(xMin,xMax,yMin,yMax)
    # screen: > = x, v = y
    screen=np.full((yMax+1-yMin,xMax+1-xMin)," ")
    for loc in oxygenMap:
        screen[loc[1]-yMin,loc[0]-xMin] = "O"
    outputStr=''
    for j,row in enumerate(screen):
        for elem in row:
            outputStr += elem
        outputStr += '\n'
    print(outputStr)

def findInitialOxygenPosition(locationsMap):
    for loc in locationsMap:
        if locationsMap[loc] == 2:
            return loc


def getMinutesForOxygenInfusion(locationsMap):
    oxygenMap=[]
    activePositions=[np.array(list(findInitialOxygenPosition(locationsMap)))]
    nextPositions=[]
    minutes = 0
    while True:
        #print("Current distance: ", currentDistance)
        #print(distanceMap)
        #print(activePositions)
        for pos in activePositions:
            if tuple(pos) in oxygenMap:
                # already with oxygen
                continue
                
            oxygenMap.append(tuple(pos))
            
            neighbors = getValidNeighborsList(pos,locationsMap)
            for neighbor in neighbors:
                nextPositions.append(np.array(list(neighbor)))
        
        if len(nextPositions) == 0:
            # converged
            visualizeOxygenMap(oxygenMap)
            return minutes - 1
        
        minutes += 1
        # swap
        activePositions,nextPositions = nextPositions,[]
        

In [104]:
minutes=getMinutesForOxygenInfusion(currentLocations)

                                         
 OOO OOOOOOO OOOOOOOOOOOOOOOOOOO OOOOOOO 
 O O O   O O O       O         O       O 
 O OOO O O O O OOO OOO OOOOOOO OOO OOOOO 
 O     O O O O O O O   O         O O   O 
 OOO OOO O O O O OOO OOOOOOOOO OOO O OOO 
   O O   O O   O             O O   O O   
 OOO O OOO O OOO OOOOOOO OOOOO OOOOO O O 
 O   O O   O O O O     O O           O O 
 OOOOO O OOOOO O O OOO O OOOOOOOOOOO O O 
     O O       O O   O O O       O   O O 
 O OOO OOO OOO O O OOO O OOOOOOO OOO OOO 
 O O     O O O   O O   O       O   O   O 
 OOO OOO OOO OOO O O OOO OOOOO O O OOO O 
 O   O         O O O O   O O   O O   O O 
 OOOOO OOOOOOOOO O O OOO O OOO O O OOO O 
     O O   O     O O   O     O O O O   O 
 O OOO O OOO OOO O OOO OOO OOO OOO OOOOO 
 O O   O     O O O   O   O O O   O       
 O O O OOOOOOO OOO OOO OOO O O OOO OOOOO 
 O O O             O   O   O   O   O   O 
 O OOOOOOO OOOOOOOOO O O OOOOO O OOO O O 
 O       O O       O O O   O O O O   O O 
 O OOOOO OOO OOO OOO O OOO O OOO O

In [105]:
print(minutes)

406
